### Neural Net LSTM Model Supervised Learning step by step

### Import libraries

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pp
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, LSTM
from keras.models import Sequential
from datetime import datetime

### Step 1: Load the Data

In [7]:
folder = os.getcwd()
csvFile = os.path.dirname(folder) + '\\Stocks_daily\\BA.csv'

data_df = pd.read_csv(csvFile)

data_df.index = pd.to_datetime( data_df.index, format='%Y-%m-%d' )
data_df.sort_index( inplace=True )

### Step 2: Split the Data into training and validation sets

In [8]:
X = data_df[['Open', 'High', 'Low', 'Close', 'Volume']]
X = X.iloc[:-21]
y_1_week = data_df['Close'].iloc[7:-14]
y_2_weeks = data_df['Close'].iloc[14:-7]
y_3_weeks = data_df['Close'].iloc[21:]

y = pd.DataFrame({'Close_1_Week': y_1_week.values, 'Close_2_Weeks': y_2_weeks.values, 'Close_3_Weeks': y_3_weeks.values}, index=None)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, shuffle = False)

### Step 3: Normalize Data

In [9]:
scaler = MinMaxScaler()
X_trainScaled = scaler.fit_transform(X_train)
X_valScaled = scaler.fit_transform(X_val)
y_trainScaled = scaler.fit_transform(y_train)
y_valScaled = scaler.fit_transform(y_val)


### Step 4: Create and Train the Model

In [10]:
X_trainScaled = X_trainScaled.reshape(X_trainScaled.shape[0],X_trainScaled.shape[1],1)
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(X_trainScaled.shape[1], X_trainScaled.shape[2])))
model.add(Dense(64, activation='relu'))
model.add(Dense(3))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_trainScaled.reshape(X_trainScaled.shape[0], X_trainScaled.shape[1], 1), y_trainScaled, 
          epochs=100, batch_size=32, validation_data=(X_valScaled.reshape(X_valScaled.shape[0], X_valScaled.shape[1], 1), y_valScaled))

Epoch 1/100
130/130 [==============================] - 1s 5ms/step - loss: 0.0173 - val_loss: 0.0055
Epoch 2/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0014 - val_loss: 0.0053
Epoch 3/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0014 - val_loss: 0.0056
Epoch 4/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0014 - val_loss: 0.0054
Epoch 5/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0014 - val_loss: 0.0053
Epoch 6/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0015 - val_loss: 0.0051
Epoch 7/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0013 - val_loss: 0.0053
Epoch 8/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0013 - val_loss: 0.0051
Epoch 9/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0013 - val_loss: 0.0053
Epoch 10/100
130/130 [==============================] - 1s 4ms/step - loss: 0.0013 - val_lo

### Step 5: Predict using the Trained Model

In [21]:
y_pred_scaled = model.predict(X_valScaled.reshape(X_valScaled.shape[0], X_valScaled.shape[1], 1))
y_pred = scaler.inverse_transform(y_pred_scaled)
y_val1 = y_val['Close_1_Week'].tolist()
X_close = X_val['Close'].tolist()
buyReal=False
buyReal_dic={'buy':[],'sell':[]}
buyTrain = False
buyTrain_dic = {'buy':[],'sell':[]}
for i in range(0,len(X_close)):
    # print(y_1_week_train_actual[i],X_close_actual[i])
    if y_val1[i] > X_close[i]:
        if buyReal == True:
            pass
        else:
            buyReal == True
            buyReal_dic['buy'].append(X_close[i])
    if y_val1[i] < X_close[i]:
        if buyReal == False:
            pass
        else:
            buyReal = False
            buyReal_dic['sell'].append(X_close[i])


    if y_pred[i,0] > X_close[i]:
        if buyTrain == True:
            pass
        else:
            buyTrain == True
            buyTrain_dic['buy'].append(X_close[i])
    if y_pred[i,0] < X_close[i]:
        if buyTrain == False:
            pass
        else:
            buyTrain = False
            buyTrain_dic['sell'].append(X_close[i])

56/56 [==============================] - 0s 2ms/step


### Step 6: Plotting

In [ ]:
# for i in range(3):
#     plt.figure(figsize=(10, 6))
#     plt.subplot(2,1,1)
#     plt.plot(X_close_cur[-length_:].index, y_val_actual[:, i], label=f'Actual {i+1}-Week')
#     plt.plot(X_close_cur[-length_:].index, y_pred[:, i], label=f'Predicted {i+1}-Week')
#     plt.xlabel('Time')
#     plt.ylabel('Closing Price')
#     plt.legend()
#     plt.title(f'Closing Price Prediction {i+1}-Weeks Ahead')
#     plt.subplot(2,1,2)
#     plt.plot( X_close_cur[-length_:].index, y_val_actual[:, i] - y_pred[:, i] )
#     plt.show()